<a href="https://colab.research.google.com/github/gandharvbakshi/AI-and-ML-Hackathons/blob/main/Dravya_Video_Frame_detection_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
print("Tensorflow version", tf.__version__)
from tensorflow import keras
print("Keras version", keras.__version__)
import matplotlib.pyplot as plt
from functools import partial
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np

np.random.seed(42) 
tf.random.set_seed(42)

import os
import pandas as pd

import cv2 
print("CV2 version", cv2.__version__)

import warnings
warnings.filterwarnings('ignore')

Tensorflow version 2.6.0
Keras version 2.6.0
CV2 version 4.1.2


In [101]:
dataDir = "/content/drive/MyDrive/AI ML Projects/Dravya/dataset"
trainDataRaw = pd.read_csv("/content/drive/MyDrive/AI ML Projects/Dravya/dataset/train.csv")
testDataRaw = pd.read_csv("/content/drive/MyDrive/AI ML Projects/Dravya/dataset/test.csv")


In [28]:
testDataRaw.tail(2)

,Frame_ID
111,Test_111.jpg
112,Test_112.jpg


In [102]:
import cv2
import os
  
# Function to extract frames
def FrameCapture(dataDir, path, stop_count, name):
      
    # Path to video file
    vidObj = cv2.VideoCapture(path)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print("Frames per second: ",fps)
    # Used as counter variable
    count = 0
  
    # checks whether frames were extracted
    success = 1
  
    while success:
  
        # vidObj object calls read
        # function extract frames
        vidObj.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) 
        success, image = vidObj.read()
        if (success == False):
          break
        if (count > stop_count):
          break
        if os.path.isfile(os.path.join(dataDir, name + "_%d.jpg" % count)):
          os.remove(os.path.join(dataDir, name + "_%d.jpg" % count)) 
        # Saves the frames with frame-count
        cv2.imwrite(os.path.join(dataDir, name + "_%d.jpg" % count), image)
  
        count += 1

In [115]:
def FrameCounter(path):
      
    # Path to video file
    vidObj = cv2.VideoCapture(path)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print("Frames per second: ",fps)
    # Used as counter variable
    count = 0
  
    # checks whether frames were extracted
    success = 1
  
    while success:
  
        # vidObj object calls read
        # function extract frames
        vidObj.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) 
        success, image = vidObj.read()
        if (success == False):
          break
        #if (count > stop_count):
        #  break

        count += 1
    return count

In [103]:
trainVideoPath = os.path.join(dataDir, 'train.mp4')
testVideoPath = os.path.join(dataDir, 'test.mp4')
#os.mkdir(os.path.join(dataDir, "train"))
trainImageDir = os.path.join(dataDir, "train")
#os.mkdir(os.path.join(dataDir, "test"))
testImageDir = os.path.join(dataDir, "test")
FrameCapture(trainImageDir, trainVideoPath, 407, "Frame")
FrameCapture(testImageDir, testVideoPath, 112, "Test")

Frames per second:  29.97002997002997
Frames per second:  24.0


In [117]:
FrameCounter(testVideoPath)

Frames per second:  24.0


113

In [104]:
from sklearn.preprocessing import LabelEncoder
le_class = LabelEncoder()
trainDataRaw['Animal'] = le_class.fit_transform(trainDataRaw['Animal'])
trainDataRaw.head(2)

,Frame_ID,Animal
0,Frame_0.jpg,5
1,Frame_1.jpg,5


In [105]:
#Current Image size - 320 * 240 ; new size - 224 * 224 
IMG_SIZE = 224 
X = [] 
y = [] 
i = 0

for img in trainDataRaw["Frame_ID"].tolist():
  if (i % 1000 == 0):
    print(i, "th image done")
  img_array = cv2.imread(os.path.join(trainImageDir,img ))
  #fsa.append(img_array)
  i = i + 1
  new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
  X.append(new_array)
  #img_array = cv2.normalize(img_array, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  #X_scaled = np.append(X_scaled, img_array) #.append()
  y.append(trainDataRaw.loc[trainDataRaw["Frame_ID"]==img, 'Animal'])
  #plt.imshow(img_array)
  #plt.show()
  #break
X = np.array(X)
#X_scaled = np.array(X_scaled)
y = np.array(y)  

0 th image done


In [106]:
i = 0
X_test = []

for img in testDataRaw["Frame_ID"].tolist():
  if (i % 1000 == 0):
    print(i, "th image done")
  img_array = cv2.imread(os.path.join(testImageDir,img ))
  if (os.path.isfile(os.path.join(testImageDir,img)) == False):
    print("File not found: ", testImageDir, img)

  i = i + 1
  new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
  X_test.append(new_array)
X_test = np.array(X_test)


0 th image done


In [107]:
y = pd.concat((trainDataRaw, pd.get_dummies(trainDataRaw['Animal'], prefix = 'Animal_')), axis=1).drop(['Frame_ID', 'Animal'],axis = 1).to_numpy()
y.shape

(408, 13)

In [108]:
print(X.shape, y.shape, X_test.shape)

(408, 224, 224, 3) (408, 13) (113, 224, 224, 3)


In [121]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions

#from keras.applications.resnet50 import ResNet50
#from keras.applications.resnet50 import preprocess_input
#from keras.applications.resnet50 import decode_predictions

from keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#import efficientnet.keras as efn
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

def VGG16_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=13, activation='softmax')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def Inception_model():
  base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=13, activation='softmax')
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def Inception_trainable_model():
  base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
  #base_model.trainable = True
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=13, activation='softmax') 
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model


def VGG16_trainable_true_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  #base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=13, activation='softmax')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def VGG16_slow_learn_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.000001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def VGG16_fast_learn_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.0001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def VGG16_dropout_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def VGG16_deeper_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def VGG16_deeper_dropout_model():
  base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def ResNet50_model():
  base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False
  model = keras.models.Sequential([base_model,
                                  keras.layers.GlobalAveragePooling2D(),
                                  #keras.layers.Flatten(),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(1024, activation='relu'),
                                  keras.layers.Dropout(0.2),
                                  keras.layers.Dense(1024, activation='relu'),
                                  #keras.layers.Dropout(0.2),
                                  keras.layers.Dense(units=1, activation='sigmoid')                                     
                                  ])
  base_learning_rate = 0.00001
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])  
  return model

def variable_model(FILTERS = 64,KERNEL_SIZE = 7):
  # create model
  DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=KERNEL_SIZE, activation='relu', padding="SAME")
  model = keras.models.Sequential([
    DefaultConv2D(filters=FILTERS, kernel_size=KERNEL_SIZE, input_shape=[224, 224, 3]), 
    keras.layers.MaxPooling2D(pool_size=2), 
    DefaultConv2D(filters=128),  
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='sigmoid')
])               
  model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])    
  return model

def old_model():
  # create model
  DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, activation='relu', padding="SAME")
  model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[224, 224, 3]), 
    keras.layers.MaxPooling2D(pool_size=2), 
    DefaultConv2D(filters=128),  
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=13, activation='softmax')
])               
  model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])    
  return model



In [110]:
#X = X.astype('float64')
#X_test = X_test.astype('float64')
#X /= 255
#X_test /= 255
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_final = X_test
print(len(x_train), "train +", len(x_test), "test")


326 train + 82 test


In [61]:
y_test
ind = np.argmax(y_test, axis= 1)#, y_test.shape)
ind

array([ 6,  0, 12,  9,  4,  6, 10,  1,  1, 12, 12,  6,  2, 12,  7,  7,  7,
        0, 12,  7,  2, 12,  2,  0,  2,  8,  1,  0,  7, 12, 10,  3,  5, 10,
       12, 12,  2,  8,  1,  9,  8, 12,  7, 12,  3,  8,  9, 12,  7, 10,  7,
       12, 12,  4, 12,  4,  6,  6,  4, 12,  2,  7,  9,  7,  1,  7, 12,  3,
        9,  2,  0,  1,  6,  7,  7, 12,  4,  1, 12,  1,  4,  1])

In [122]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

batch_size = 32

# defining the path to save the model
best_model_path = os.path.join(dataDir, 'best_model.hdf5')

model = old_model() #variable_model(32, 3) #old_model()
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    ModelCheckpoint(best_model_path, monitor='val_accuracy', verbose = 1, save_best_only = True, mode='max')
]

trial = model.fit(x_train,y_train,  epochs=131, validation_data = (x_test, y_test), callbacks=my_callbacks) #batch_size=batch_size,                      

model.load_weights(best_model_path)
y_bm_keras = model.predict(x_test)
y_test_class = np.argmax(y_test, axis= 1)
y_bm_keras_class = np.argmax(y_bm_keras, axis= 1)
print("Train Accuracy Score " , f1_score(y_test_class, y_bm_keras_class, average='micro') )


Epoch 1/131
11/11 [==============================] - 3s 134ms/step - loss: 60.8164 - accuracy: 0.1043 - val_loss: 2.4656 - val_accuracy: 0.0976

Epoch 00001: val_accuracy improved from -inf to 0.09756, saving model to /content/drive/MyDrive/AI ML Projects/Dravya/dataset/best_model.hdf5
Epoch 2/131
11/11 [==============================] - 1s 109ms/step - loss: 2.5410 - accuracy: 0.2086 - val_loss: 2.4816 - val_accuracy: 0.2317

Epoch 00002: val_accuracy improved from 0.09756 to 0.23171, saving model to /content/drive/MyDrive/AI ML Projects/Dravya/dataset/best_model.hdf5
Epoch 3/131
11/11 [==============================] - 1s 109ms/step - loss: 2.9118 - accuracy: 0.2209 - val_loss: 3.0807 - val_accuracy: 0.2317

Epoch 00003: val_accuracy did not improve from 0.23171
Epoch 4/131
11/11 [==============================] - 1s 108ms/step - loss: 2.6693 - accuracy: 0.2055 - val_loss: 2.3419 - val_accuracy: 0.2317

Epoch 00004: val_accuracy did not improve from 0.23171
Epoch 5/131
11/11 [=======

In [71]:
y_bm_keras_class

array([ 7,  4, 12,  1, 10,  1,  7,  1,  1, 12,  6, 12,  1, 12, 12,  7, 12,
       12,  2, 12,  7,  6,  1,  1,  4,  9, 12,  0,  0,  6,  1,  1,  2, 12,
        7, 12,  7, 12,  2,  2,  6,  7,  7, 10, 12,  0,  4,  4, 12,  5, 12,
       12,  7, 12,  8,  1, 12,  1,  1, 12,  7, 10, 10,  7,  0,  7,  1,  2,
        3,  7,  1, 10,  7,  6,  4,  9, 10, 12,  1, 12,  0,  2])

In [123]:
y_bm_final = model.predict(x_final)
y_bm_final = np.argmax(y_bm_final, axis= 1)
y_bm_final = le_class.inverse_transform(y_bm_final)
testDataSub = testDataRaw.copy()
testDataSub['Animal'] = y_bm_final
testDataSub.to_csv(os.path.join(dataDir, 'submission4 CNN.csv'), index = False)